## Deliverable 2. Create a Customer Travel Destinations Map.

In [111]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import google_map_api_key

# Configure gmaps API key
gmaps.configure(api_key=google_map_api_key)

In [112]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City ID,City,Country,Latitude,Longitude,Max Temperature,Humidity,Percent Clouds,Wind Speed,Description,Date
0,0,Avarua,CK,-21.2078,-159.7750,73.45,73,90,6.91,overcast clouds,2021-06-13 01:47:11
1,1,Vaini,TO,-21.2000,-175.2000,77.16,69,40,12.66,scattered clouds,2021-06-13 01:47:11
2,2,Albany,US,42.6001,-73.9662,66.07,82,15,2.30,few clouds,2021-06-13 01:42:32
3,3,Ahuimanu,US,21.4447,-157.8378,87.51,56,40,13.80,scattered clouds,2021-06-13 01:47:12
4,4,Iquique,CL,-20.2208,-70.1431,62.64,78,5,3.20,clear sky,2021-06-13 01:47:12


In [113]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
user_min_temp = float(input("What temperature minimum do you prefer?"))
user_max_temp = float(input("What temperature maximum do you prefer?"))

In [114]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
user_pref_cities_df = city_data_df.loc[(city_data_df["Max Temperature"] <= user_max_temp) & (city_data_df["Max Temperature"] >= user_min_temp)]

In [115]:
# 4a. Determine if there are any empty rows.
user_pref_cities_df.count()

City ID            493
City               493
Country            486
Latitude           493
Longitude          493
Max Temperature    493
Humidity           493
Percent Clouds     493
Wind Speed         493
Description        493
Date               493
dtype: int64

In [116]:
# 4a-1. Further examination for empty rows/data.
series = pd.isnull(user_pref_cities_df["Country"])
user_pref_cities_df[series]

,City ID,City,Country,Latitude,Longitude,Max Temperature,Humidity,Percent Clouds,Wind Speed,Description,Date
299,299,Oranjemund,NaN,-28.5500,16.4333,56.59,84,14,6.51,few clouds,2021-06-13 02:06:51
377,377,Keetmanshoop,NaN,-26.5833,18.1333,61.16,30,28,5.66,scattered clouds,2021-06-13 02:13:17
455,455,Luderitz,NaN,-26.6481,15.1594,57.43,83,11,2.28,few clouds,2021-06-13 02:16:42
492,492,Mariental,NaN,-24.6333,17.9667,55.90,31,55,7.96,broken clouds,2021-06-13 02:19:54
511,511,Opuwo,NaN,-18.0607,13.8400,56.26,33,3,7.38,clear sky,2021-06-13 02:23:00
563,563,Ongandjera,NaN,-17.8833,15.0667,56.66,33,10,6.76,clear sky,2021-06-13 02:26:19
602,602,Khorixas,NaN,-20.3667,14.9667,60.96,29,16,8.99,few clouds,2021-06-13 02:29:31


In [117]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
pref_cities_clean_df = user_pref_cities_df.dropna()
pref_cities_clean_df.count()

City ID            486
City               486
Country            486
Latitude           486
Longitude          486
Max Temperature    486
Humidity           486
Percent Clouds     486
Wind Speed         486
Description        486
Date               486
dtype: int64

In [118]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = pref_cities_clean_df[["City", "Country", "Max Temperature", "Description", "Latitude", "Longitude"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temperature,Description,Latitude,Longitude,Hotel Name
0,Avarua,CK,73.45,overcast clouds,-21.2078,-159.7750,
1,Vaini,TO,77.16,scattered clouds,-21.2000,-175.2000,
2,Albany,US,66.07,few clouds,42.6001,-73.9662,
4,Iquique,CL,62.64,clear sky,-20.2208,-70.1431,
5,Hobyo,SO,77.14,overcast clouds,5.3505,48.5268,
6,Pisco,PE,64.45,overcast clouds,-13.7000,-76.2167,
7,Atuona,PF,79.09,clear sky,-9.8000,-139.0333,
8,Tairua,NZ,62.10,overcast clouds,-37.0167,175.8500,
11,Bandarbeyla,SO,79.16,broken clouds,9.4942,50.8122,
15,Yellowknife,CA,61.32,few clouds,62.4560,-114.3525,


In [119]:
# scratch for verifying my request without running thru the entire df...
# params = {
#     "radius": 5000,
#     "type": "lodging",
#     "key": google_map_api_key
# }
# latitude = -21.2078
# longitude = -159.7750
# params["location"] = latitude, longitude
# # params
# places_url = "https://maps.googleapis.com/maps/api/place/textsearch/json?"
# result = requests.get(places_url, params).json()
# result["results"][0]["name"]

In [120]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": google_map_api_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    latitude = row["Latitude"]
    longitude = row["Longitude"]
    params["location"] = f"{latitude},{longitude}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    # Directions API or Places API? Seems like Places would be the better choice...
    # Going with Places API and NEARBY search for now...
    places_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search.
    hotel_data = requests.get(places_url, params=params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_data["results"][0]["name"]
    except (IndexError):
        hotel_df.loc[index, "Hotel Name"] = None
        print(f"No hotels found for index: {index}, skipping.")
hotel_df.head(10)

No hotels found for index: 2, skipping.
No hotels found for index: 80, skipping.
No hotels found for index: 101, skipping.
No hotels found for index: 120, skipping.
No hotels found for index: 138, skipping.
No hotels found for index: 142, skipping.
No hotels found for index: 174, skipping.
No hotels found for index: 177, skipping.
No hotels found for index: 207, skipping.
No hotels found for index: 217, skipping.
No hotels found for index: 226, skipping.
No hotels found for index: 239, skipping.
No hotels found for index: 247, skipping.
No hotels found for index: 248, skipping.
No hotels found for index: 253, skipping.
No hotels found for index: 267, skipping.
No hotels found for index: 286, skipping.
No hotels found for index: 295, skipping.
No hotels found for index: 306, skipping.
No hotels found for index: 319, skipping.
No hotels found for index: 366, skipping.
No hotels found for index: 385, skipping.
No hotels found for index: 388, skipping.
No hotels found for index: 408, skipp

,City,Country,Max Temperature,Description,Latitude,Longitude,Hotel Name
0,Avarua,CK,73.45,overcast clouds,-21.2078,-159.7750,Paradise Inn
1,Vaini,TO,77.16,scattered clouds,-21.2000,-175.2000,Keleti Beach Resort
2,Albany,US,66.07,few clouds,42.6001,-73.9662,None
4,Iquique,CL,62.64,clear sky,-20.2208,-70.1431,Backpacker`s Hostel Iquique
5,Hobyo,SO,77.14,overcast clouds,5.3505,48.5268,Halane Hotel and Restaurent
6,Pisco,PE,64.45,overcast clouds,-13.7000,-76.2167,Hotel San Isidro Oficial
7,Atuona,PF,79.09,clear sky,-9.8000,-139.0333,Villa Enata
8,Tairua,NZ,62.10,overcast clouds,-37.0167,175.8500,Pacific Harbour Villas
11,Bandarbeyla,SO,79.16,broken clouds,9.4942,50.8122,JABIR HOTEL
15,Yellowknife,CA,61.32,few clouds,62.4560,-114.3525,The Explorer Hotel


In [121]:
# examine counts looking for nulls
hotel_df.count()

City               486
Country            486
Max Temperature    486
Description        486
Latitude           486
Longitude          486
Hotel Name         446
dtype: int64

In [122]:
# filter to examine potential null values
hotel_series = pd.isnull(hotel_df["Hotel Name"])
hotel_df[hotel_series]

,City,Country,Max Temperature,Description,Latitude,Longitude,Hotel Name
2,Albany,US,66.07,few clouds,42.6001,-73.9662,None
80,Lusambo,CD,68.49,clear sky,-4.9667,23.4500,None
101,Qarqin,AF,84.24,clear sky,37.4185,66.0436,None
120,Srednekolymsk,RU,69.98,overcast clouds,67.4500,153.6833,None
138,Lola,GN,70.57,light rain,7.8000,-8.5333,None
142,Vagay,RU,64.54,overcast clouds,57.9375,69.0217,None
174,Abu Samrah,SY,64.74,clear sky,35.3029,37.1841,None
177,Boda,CF,70.52,light rain,4.3189,17.4695,None
207,Fonte Boa,BR,73.13,overcast clouds,-2.4959,-66.2662,None
217,Evensk,RU,55.85,clear sky,61.9500,159.2333,None


In [123]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [124]:
# Re-examine for null values
clean_hotel_df.count()

City               446
Country            446
Max Temperature    446
Description        446
Latitude           446
Longitude          446
Hotel Name         446
dtype: int64

In [125]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [126]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Description} and {Max Temperature} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Latitude", "Longitude"]]

In [133]:
# 11a. Add a marker layer for each city to the map. 
map_markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
# 11b. Display the figure
# Create the figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.85)
fig.add_layer(map_markers)
fig

Figure(layout=FigureLayout(height='420px'))